In [1]:
from pathlib import Path
import pdfplumber
import pandas as pd
import re

In [27]:

def scrapPDF(ruta, encabezados_objetivo):
    # Extraer tablas por página
    tablas_por_pagina = []
    
    with pdfplumber.open(ruta) as pdf:
        for pagina in pdf.pages:
            tablas_pagina = []
            for tabla in pagina.extract_tables():
                if tabla and len(tabla) > 1:  
                    tablas_pagina.append(tabla)
            tablas_por_pagina.append(tablas_pagina)


    # Función auxiliar para verificar si una tabla tiene los encabezados objetivo
    def tiene_encabezados_objetivo(tabla):
        if not tabla or len(tabla) == 0:
            return False
        encabezados = [str(celda).strip().lower() for celda in tabla[0] if celda]
        return encabezados_objetivo.issubset(set(encabezados))

    # Buscar la tabla con los encabezados objetivo
    tabla_encontrada = None
    indice_pagina = -1
    indice_tabla = -1

    for i, tablas_pag in enumerate(tablas_por_pagina):
        for j, tabla in enumerate(tablas_pag):
            if tiene_encabezados_objetivo(tabla):
                tabla_encontrada = tabla
                indice_pagina = i
                indice_tabla = j
                break
        if tabla_encontrada:
            break

    if not tabla_encontrada:
        return pd.DataFrame()

    # Combinar con tablas de páginas siguientes si es necesario
    filas_combinadas = list(tabla_encontrada)
    pagina_actual = indice_pagina

    while True:
        tablas_pag_actual = tablas_por_pagina[pagina_actual]
        
        # Verificar si la tabla actual es la última de la página
        if indice_tabla != len(tablas_pag_actual) - 1:
            # No es la última tabla de la página, terminar
            break
        
        # Verificar si hay una página siguiente
        if pagina_actual + 1 >= len(tablas_por_pagina):
            # No hay más páginas, terminar
            break
        
        # Verificar si la página siguiente tiene tablas
        tablas_pag_siguiente = tablas_por_pagina[pagina_actual + 1]
        if not tablas_pag_siguiente:
            # No hay tablas en la siguiente página, terminar
            break
        
        # Obtener la primera tabla de la página siguiente
        primera_tabla_siguiente = tablas_pag_siguiente[0]
        
        # Verificar si la primera tabla de la siguiente página tiene encabezados
        if tiene_encabezados_objetivo(primera_tabla_siguiente):
            # Tiene encabezados, no combinar
            break
        
        # Combinar: agregar las filas de la primera tabla de la siguiente página
        # (sin incluir la primera fila si parece un encabezado)
        filas_combinadas.extend(primera_tabla_siguiente)
        
        # Actualizar para la siguiente iteración
        pagina_actual += 1
        indice_tabla = 0

    # Crear DataFrame con las filas combinadas
    df = pd.DataFrame(filas_combinadas[1:], columns=filas_combinadas[0])
    df["Nombre"] = df["Nombre"].replace("\\n", " ")
    df = df[~df["Nombre"].str.contains(r"^https?://", na=False)]
    
    return df

In [32]:
df = scrapPDF("../GA_61IW_615000237_1S_2024-25.pdf", {"nombre", "despacho", "correo electrónico"})

In [34]:
df.head(6)

,Nombre,Despacho,Correo electrónico,Horario de tutorías\n*
0,Felix Jose Fuentes Hurtado,,felix.fuentes@upm.es,Sin horario.
1,Santiago Alonso Villaverde,,santiago.alonso@upm.es,Sin horario.
2,Edgar Talavera Muñoz,,e.talavera@upm.es,Sin horario.
3,Fernando Ortega Requena\n(Coordinador/a),,fernando.ortega@upm.es,- -
4,Maria Gema Bello Orgaz,,gema.borgaz@upm.es,Sin horario.
5,Maria Angeles Mahillo\nGarcia,,mariaangeles.mahillo@upm.\nes,Sin horario.
